# Spark Apache (семинары)
### Урок 3. Инструменты работы и визуализации ч.2
**Условие:** есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),  
("2023-11-21", "Electronics", 110, 13000),  
("2023-11-22", "Electronics", 105, 12500),  
("2023-11-20", "Clothing", 300, 15000),  
("2023-11-21", "Clothing", 280, 14000),  
("2023-11-22", "Clothing", 320, 16000),  
("2023-11-20", "Books", 150, 9000),  
("2023-11-21", "Books", 200, 12000),  
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [13]:
# Инициализация Spark сессии
spark = SparkSession.builder \
    .appName("MeanRevenue") \
    .getOrCreate()

In [3]:
# Определяем схему датафрейма
df_schema = StructType([
    StructField("date", StringType(), True),
    StructField("category", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("revenue", IntegerType(), True)
])

In [4]:
# Создаем данные
sales = [
        ("2023-11-20", "Electronics", 100, 12000),  
        ("2023-11-21", "Electronics", 110, 13000),  
        ("2023-11-22", "Electronics", 105, 12500),  
        ("2023-11-20", "Clothing", 300, 15000),  
        ("2023-11-21", "Clothing", 280, 14000),  
        ("2023-11-22", "Clothing", 320, 16000),  
        ("2023-11-20", "Books", 150, 9000),  
        ("2023-11-21", "Books", 200, 12000),  
        ("2023-11-22", "Books", 180, 10000)
]

In [ ]:
# Вариант 1. Создаем RDD датафрейм из данных по схеме
sales_df = spark.createDataFrame(sales, schema=df_schema)

print("Информация о продажах:")
sales_df.show()

Список сотрудников и менеджеров:
+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [19]:
# Вариант 2. Читаем данные из CSV файла
sales_df=spark.read.load('G:\Мой диск\GeekBrains\Spark\Homework3\sales.csv', 
                           format="csv", 
                           sep=";", 
                           header="true", 
                           Infer_schema=True)   # 

# Преобразование поля date в тип данных DateType
sales_df = sales_df.withColumn("date", F.to_date("date"))

print("Информация о продажах:")
sales_df.show()

Информация о продажах:
+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [25]:
# Определение оконной спецификации
window_spec = Window.partitionBy("category") 

# Оконная функция: вычисление средней выручки от продаж для каждой категории продукта
avg_revenue = F.avg("revenue").over(window_spec)

# Применение оконной функции и приведение к double
result_df = sales_df.withColumn("avg_revenue", avg_revenue.cast(DoubleType()))

# Отображение результатов: используем distinct() для исключения повторов 
result_df.select(["category", "avg_revenue"]).distinct().show()

+-----------+------------------+
|   category|       avg_revenue|
+-----------+------------------+
|      Books|10333.333333333334|
|   Clothing|           15000.0|
|Electronics|           12500.0|
+-----------+------------------+



In [ ]:
# Применение функции pivot
pivot_df = sales_df.groupBy("category").pivot("date").agg(F.avg("revenue")).orderBy("category")

# Вывод средних значений выручки от продаж по категориям за соответствующий день
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+



In [12]:
spark.stop()